# **Memory & Chain**

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [2]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### 2) 라이브러리 로딩

In [3]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [4]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.Memory**

### (1) 대화 기록 저장하기

In [6]:
from langchain.memory import ConversationBufferMemory

In [7]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True)

# 저장
memory.save_context({"input": "안녕하세요!"},
                    {"output": "안녕하세요! 어떻게 도와드릴까요?"})

memory.save_context({"input": "메일을 써야하는데 도와줘"},
                    {"output": "누구에게 보내는 어떤 메일인가요?"})

# 현재 담겨 있는 메모리 내용 전체 확인
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요!'),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?'),
  HumanMessage(content='메일을 써야하는데 도와줘'),
  AIMessage(content='누구에게 보내는 어떤 메일인가요?')]}

### (2) GPT와 대화 기록 저장하기

#### 1) 대화 준비

* 03 파일에서 생성한 database를 지정하여 사용.

In [8]:
k = 3

# Chroma 데이터베이스 인스턴스
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = path + "db2", embedding_function = embeddings)

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# RetrievalQA 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

#### 2) 대화 시도 및 기록

In [9]:
# 질문 답변1
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)

memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다음과 같습니다:\n1. 모델 유출: 생성된 AI 모델이 불법적으로 복제되어 유출될 수 있습니다.\n2. 해킹 및 악용: 해커들이 AI 시스템을 침입하여 악의적인 목적으로 활용할 수 있습니다.\n3. 개인정보 노출: AI 시스템이 개인정보를 처리하는 경우, 해당 정보가 노출될 위험이 있습니다.\n4. 혐오 발언 및 차별: 생성형 AI가 부적절한 콘텐츠를 생성하거나 혐오 발언을 확산시킬 수 있습니다.\n5. 데이터 조작: 악의적인 공격자가 AI 모델을 속이기 위해 데이터를 조작할 수 있습니다.\n\n이 외에도 다양한 보안 위협이 존재할 수 있으며, 이를 방지하기 위해 적절한 보안 대책이 필요합니다.')]}

#### 3) 이어지는 질문과 답변

In [10]:
# 질문 답변2
query = "훈련 데이터나 가중치를 오염시키는게 무슨 의미야?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다음과 같습니다:\n1. 모델 유출: 생성된 AI 모델이 불법적으로 복제되어 유출될 수 있습니다.\n2. 해킹 및 악용: 해커들이 AI 시스템을 침입하여 악의적인 목적으로 활용할 수 있습니다.\n3. 개인정보 노출: AI 시스템이 개인정보를 처리하는 경우, 해당 정보가 노출될 위험이 있습니다.\n4. 혐오 발언 및 차별: 생성형 AI가 부적절한 콘텐츠를 생성하거나 혐오 발언을 확산시킬 수 있습니다.\n5. 데이터 조작: 악의적인 공격자가 AI 모델을 속이기 위해 데이터를 조작할 수 있습니다.\n\n이 외에도 다양한 보안 위협이 존재할 수 있으며, 이를 방지하기 위해 적절한 보안 대책이 필요합니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 해당 데이터나 모델의 정보를 부정확하게 만들거나 변형시키는 것을 의미합니다. 이는 모델의 성능을 저하시키거나 잘못된 결론을 이끌어낼 수 있습니다. 따라서 데이터나 가중치의 오염을 방지하기 위해 데이터의 품질을 유지하고 모델의 안정성을 고려하는 것이 중요합니다.')]}

In [11]:
# 질문 답변3
query = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다음과 같습니다:\n1. 모델 유출: 생성된 AI 모델이 불법적으로 복제되어 유출될 수 있습니다.\n2. 해킹 및 악용: 해커들이 AI 시스템을 침입하여 악의적인 목적으로 활용할 수 있습니다.\n3. 개인정보 노출: AI 시스템이 개인정보를 처리하는 경우, 해당 정보가 노출될 위험이 있습니다.\n4. 혐오 발언 및 차별: 생성형 AI가 부적절한 콘텐츠를 생성하거나 혐오 발언을 확산시킬 수 있습니다.\n5. 데이터 조작: 악의적인 공격자가 AI 모델을 속이기 위해 데이터를 조작할 수 있습니다.\n\n이 외에도 다양한 보안 위협이 존재할 수 있으며, 이를 방지하기 위해 적절한 보안 대책이 필요합니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 해당 데이터나 모델의 정보를 부정확하게 만들거나 변형시키는 것을 의미합니다. 이는 모델의 성능을 저하시키거나 잘못된 결론을 이끌어낼 수 있습니다. 따라서 데이터나 가중치의 오염을 방지하기 위해 데이터의 품질을 유지하고 모델의 안정성을 고려하는 것이 중요합니다.'),
  HumanMessage(content='이를 방지하기 위해 어떻게 해야 해?'),
  AIMessage(content='어떤 문제를 방지하려는지 구체적인 내용을 알려주시면 더 정확한 도움을 드릴 수 있습니다.')]}

* 훈련 데이터 오염을 방지하기 위한 대책을 물었으나, 일반적인 보안 위협 방지 대책을 이야기 함.
* 맥락을 유지하기 위해서 메시지의 내용을 프롬프트에 포함시켜야 함.
    * 이를 손쉽게 엮어주는 방법 **Chain**

## **3.Chain**

* 절차 다시 정리
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 이전 대화 내용을 메모리에서 읽어오기
    * [질문 + 유사도높은 문서 + 이전 대화내용]으로 프롬프트 구성
    * GPT에 질문하고 답변 받기
* Chain
    * 이러한 절차를 코드로 하나하나 엮고, 프롬프트를 구성하는 코드는 상당히 복잡합니다.
    * 랭체인에서 제공하는 Chain 함수를 이용하면 쉽게 구현 가능!  

### (1) Chain 함수로 연결하기
* **ConversationalRetrievalChain**

In [12]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./db2", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True,  output_key="answer")

### (2) 사용하기

#### 1) 첫번째 질문

In [13]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성된 콘텐츠를 이용한 사칭, 개인정보 누설, 악의적인 목적으로의 사용, 유해한 콘텐츠 생성 등이 있을 수 있습니다. 또한, 생성형 AI 시스템 자체가 해킹당하거나 악용될 가능성도 있습니다. 보안 전문가들은 이러한 위협을 예방하고 대비하기 위한 방안을 연구하고 있습니다.'

In [14]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성된 콘텐츠를 이용한 사칭, 개인정보 누설, 악의적인 목적으로의 사용, 유해한 콘텐츠 생성 등이 있을 수 있습니다. 또한, 생성형 AI 시스템 자체가 해킹당하거나 악용될 가능성도 있습니다. 보안 전문가들은 이러한 위협을 예방하고 대비하기 위한 방안을 연구하고 있습니다.')]}

#### 2) 두번째 질문

In [18]:
# 두번째 질문
query2 = "모델을 재학습시키면 어떤 문제가 발생되는거야?"
result = qa(query2)
result['answer']

'재학습시 모델에 발생할 수 있는 문제는 여러 가지가 있을 수 있습니다. 예를 들어, 새로운 데이터가 이전 학습 데이터와 매우 다를 경우 오버피팅(overfitting)이 발생할 수 있고, 학습 데이터가 부족한 경우 언더피팅(underfitting)이 발생할 수도 있습니다. 또한, 재학습을 위한 데이터의 품질이 낮거나 레이블링이 잘못되었을 경우 모델의 성능이 저하될 수도 있습니다. 또한, 모델이 과거 데이터에 지나치게 의존하여 새로운 트렌드나 패턴을 놓치는 경우도 발생할 수 있습니다.'

In [16]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성된 콘텐츠를 이용한 사칭, 개인정보 누설, 악의적인 목적으로의 사용, 유해한 콘텐츠 생성 등이 있을 수 있습니다. 또한, 생성형 AI 시스템 자체가 해킹당하거나 악용될 가능성도 있습니다. 보안 전문가들은 이러한 위협을 예방하고 대비하기 위한 방안을 연구하고 있습니다.'),
  HumanMessage(content='모델을 재학습시면 어떤 문제가 발생되는거야?'),
  AIMessage(content='잘못된 학습 내용을 더욱 강화할 수 있다는 것이 재학습의 위험 중 하나입니다. 또한 새로운 정보를 습득하기 어려워질 수 있고, 지루하거나 피로감을 느낄 수도 있습니다.')]}

In [ ]:
# memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
#                                   return_messages=True)

### 😀실습

* VectorDB 내용을 확인한 후, 연관된 질문으로 연속해서 물어보고 답변을 잘 하는지 확인해 봅시다.


In [19]:
# 세번째 질문
query3 = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query3)
result['answer']

'모델을 재학습시킬 때 발생할 수 있는 문제를 방지하는 방법은 다양합니다. 몇 가지 방법은 다음과 같습니다:\n\n1. **이상치 처리**: 이상치는 모델의 성능을 저하시킬 수 있으므로, 데이터에서 이상치를 식별하고 처리하는 것이 중요합니다.\n  \n2. **데이터 불균형 처리**: 클래스 불균형이 존재하는 경우, 적절한 샘플링 기술(언더샘플링, 오버샘플링)을 사용하여 데이터 불균형을 처리할 수 있습니다.\n  \n3. **하이퍼파라미터 튜닝**: 모델의 하이퍼파라미터를 조정하여 최적의 성능을 얻을 수 있습니다.\n  \n4. **교차 검증**: 모델의 일반화 성능을 높이기 위해 교차 검증을 수행하여 모델의 안정성을 확인할 수 있습니다.\n  \n5. **조기 종료**: 모델이 과적합되는 것을 방지하기 위해 조기 종료 기법을 사용할 수 있습니다.\n\n이외에도 모델 학습 중 발생할 수 있는 다양한 문제를 방지하는 방법이 있습니다.'

In [20]:
# 네번째 질문
query4 = "교차검증으로 문제를 어떻게 해결할수 있어?"
result = qa(query4)
result['answer']

'교차검증은 기계 학습 알고리즘의 일반화 성능을 평가하고 모델의 성능을 향상시키는 데 사용될 수 있습니다. 일반적으로 교차검증은 데이터를 여러 하위 집합으로 분할하여 모델을 여러 번 학습하고 평가함으로써 모델의 성능을 확인합니다. 이를 통해 모델이 새로운 데이터에 대해 얼마나 잘 일반화되는지를 측정할 수 있습니다.'

### (3) 반복문 안에서 질문답변 이어가기

In [ ]:
while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)